In [10]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules', '/tmp/tmpvvjrvgog']


In [19]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [20]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [98]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [99]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack
lora_mlm_target = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/lora-mlm-target",with_head=False)




In [100]:
from adapters import PrefixTuningConfig,LoRAConfig

#adapter_name="prefix_tuning_with_mlm_for_task"

#adapter_config = PrefixTuningConfig(flat=False)
#model.add_adapter(adapter_name, config=adapter_config)
adapter_name="lora_for_task"

adapter_config = LoRAConfig(r=8, alpha=16)
model.add_adapter(adapter_name, config=adapter_config)

fn.print_trainable_parameters(model)

trainable params: 67280442 || all params: 67280442 || trainable%: 100.0


In [101]:
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.train_adapter([adapter_name])
model.active_adapters = Stack(lora_mlm_target,adapter_name)

fn.print_trainable_parameters(model)

trainable params: 1363005 || all params: 67873341 || trainable%: 2.0081595806518497


In [102]:
model.adapter_summary(as_dict=True)

[{'name': 'lora-mlm-target',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': False,
  '%param': 0.22219650503413957},
 {'name': 'lora_for_task',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': True,
  '%param': 0.22219650503413957},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [103]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


distilbert.transformer.layer.0.attention.q_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.0.attention.q_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.0.attention.v_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.0.attention.v_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.1.attention.q_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.1.attention.q_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.1.attention.v_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.1.attention.v_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.2.attention.q_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.2.attention.q_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.2.attention.v_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.2.attention.v_lin.loras.lora_for_task.lora_B
distilbert.transformer.layer.3.attention.q_lin.loras.lora_for_task.lora_A
distilbert.transformer.layer.3.attenti

In [104]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.34615384615384615
F1 score before adaptation: 0.25271261366105136


In [105]:
reload(config)
reload(fn)
trainer = fn.train_model(model,"lora-for-task",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 1.0802, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.9885428547859192, 'eval_accuracy': 0.49566903684550745, 'eval_f1': 0.46952201666951354, 'eval_precision': 0.5808539605707265, 'eval_recall': 0.49566903684550745, 'eval_runtime': 11.2016, 'eval_samples_per_second': 690.528, 'eval_steps_per_second': 21.604, 'epoch': 0.21}
{'loss': 0.9039, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.7259854078292847, 'eval_accuracy': 0.6914027149321267, 'eval_f1': 0.6905914347523365, 'eval_precision': 0.6963944136888134, 'eval_recall': 0.6914027149321267, 'eval_runtime': 11.4663, 'eval_samples_per_second': 674.588, 'eval_steps_per_second': 21.105, 'epoch': 3.62}
{'loss': 0.727, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.7234932780265808, 'eval_accuracy': 0.692049127343245, 'eval_f1': 0.691064811717854, 'eval_precision': 0.6973325574941849, 'eval_recall': 0.692049127343245, 'eval_runtime': 11.5085, 'eval_samples_per_second': 672.112, 'eval_steps_per_second': 21.028, 'epoch': 3.84}


Overwriting existing adapter 'lora-mlm-target'.
Overwriting existing adapter 'lora_for_task'.
Overwriting existing head 'lora_for_task'


{'train_runtime': 1069.3449, 'train_samples_per_second': 280.594, 'train_steps_per_second': 8.772, 'train_loss': 0.7855583662671575, 'epoch': 4.0}


In [106]:
trainer.evaluate(test_target_data)


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.7371801733970642,
 'eval_accuracy': 0.6811740890688259,
 'eval_f1': 0.6816810492478558,
 'eval_precision': 0.6931641833533766,
 'eval_recall': 0.6811740890688259,
 'eval_runtime': 2.836,
 'eval_samples_per_second': 696.763,
 'eval_steps_per_second': 21.862,
 'epoch': 4.0}

In [107]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.6811740890688259
F1 score before adaptation: 0.6816810492478558


In [28]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name,with_head=True)